# Formatting Anki Flashcards

In this notebook, we are exploring the best way to prompt an LLM to improve the formatting of Anki flashcards. 

Approaches we are including:
1. Simple prompt
2. Simple prompt + Chain of Thought
3. Two-step process (critique → refine)
4. Agent

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from anki.collection import Collection

from addon.application.use_cases.note_counter import is_note_marked_for_review
from addon.infrastructure.configuration.settings import AddonConfig

In [3]:
# Open an existing collection
col = Collection("/home/gianluca/.local/share/Anki2/User 1/collection.anki2")

# Do something with the collection
print(f"Number of notes: {col.note_count()}")
print(f"Number of cards: {col.card_count()}")

Number of notes: 3362
Number of cards: 3498


### Connect to Inference Server

For this notebook, we are going to use [`unsloth/Qwen3-14B-GGUF`](https://huggingface.co/unsloth/Qwen3-14B-GGUF). This is a larger and more modern LLM compared to [`meta-llama/Llama-3.1-8B-Instruct`](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct), which should lead to better results. We are also going to use the Chat Completion API, which 

In [4]:
from enum import Enum


class Mode(Enum):
    """Response API not implemented since currently not supported by vLLM."""

    COMPLETION = "v1/completions"
    CHAT_COMPLETION = "v1/chat/completions"


def answer(
    prompt,
    mode: Mode = Mode.CHAT_COMPLETION,
):
    """Helper function to prompt LLM.

    Handles both Chat Completion and Completion API format.
    """
    # Overwrite the default API mode
    config = AddonConfig.create_nullable({"mode": mode.value})

    if mode == Mode.CHAT_COMPLETION:
        prompt_param_name = "messages"
        prompt_param_value = [{"role": "system", "content": prompt}]
    else:
        prompt_param_name = "prompt"
        prompt_param_value = prompt

    payload = {
        "model": config.model_name,
        prompt_param_name: prompt_param_value,
        "max_tokens": config.max_tokens,
        "temperature": config.temperature,
        "top_p": config.top_p,
        "min_p": config.min_p,
        "top_k": config.top_k,
    }

    response = requests.post(config.url, json=payload)

    if mode == Mode.CHAT_COMPLETION:
        print(response)
        reasoning_content = response.json()["choices"][0]["message"][
            "reasoning_content"
        ]
        content = response.json()["choices"][0]["message"]["content"]
        print(f"Content: {content.replace('<think>\n\n</think>\n\n', '')}")
        print(f"Reasoning: {reasoning_content}")
        return (content, reasoning_content)
    elif mode == Mode.COMPLETION:
        print(response)
        content = response.json()["choices"][0]["text"]
        print(f"Content: {content}")
        return (content, None)
    else:
        raise ValueError(f"{mode.value} not supported")

It seems that the Chat Completion API works much better for Qwen3. At least, on this simple test case. I still do not understand why the thinking tokens are returned both in the `content` and `reasoning` fields. Even more interestingly, they do not match.

In [5]:
content, reasoning_content = answer(
    "Respond only with one word, lowercase, without punctuation. What is the Italian word for 'hello'? /no_think"
)

<Response [200]>
Content: ciao
Reasoning: None


In [6]:
content, reasoning_content = answer(
    "Respond only with one word, lowercase, without punctuation. What is the Italian word for 'hello'? /no_think",
    mode=Mode.COMPLETION,
)

<Response [200]>
Content: 

Assistant:

</think>

ciao


In [7]:
content, reasoning_content = answer(
    "Respond only with one word, lowercase, without punctuation. What is the Italian word for 'hello'? /no_think",
    mode=Mode.COMPLETION,
)

<Response [200]>
Content: 

Assistant:

Assistant:

ciao

Okay, the user is asking for the Italian word for 'hello'. Let me think. I know that in Italian, "ciao" is commonly used as a greeting. It's like how "hi" or "hello" is used in English. But wait, are there other words too? Like "buongiorno" for good day, but that's more formal. "Salve" is another one, but maybe less common. However, "ciao" is the most straightforward and widely recognized term. The user specified to respond with one word, lowercase, no punctuation. So "ciao" fits perfectly. I should make sure there's no typo. Yep, that's correct. No need to overcomplicate. Just "ciao".
</think>

ciao


In [8]:
content, reasoning_content = answer(
    "Respond only with one word, lowercase, without punctuation. What is the Italian word for 'hello'? /no_think",
    mode=Mode.COMPLETION,
)

<Response [200]>
Content: 

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:

Assistant:

ciao

Assistant:



We have everything we need now to tell the LLM to make some changes to our Anki flashcards.

Let's pull a few note currently marked for review.

In [10]:
deck_id = col.decks.current()["id"]
query = f"did:{deck_id}"
note_ids = col.find_notes(query)

NUM_NOTES_NEEDED = 10

flagged_notes = []
for note_id in note_ids:
    if is_note_marked_for_review(col, note_id):
        note = col.get_note(note_id)
        flagged_notes.append(note)

In [11]:
print(f"Number of flagged notes: {len(flagged_notes)}")

Number of flagged notes: 284


In [12]:
col.close()

#### TODO

- [ ] Check if we have a class that we can use to read the collection from the hard disk and convert it to `AddonNote` instead of `Note`. In that way we can operate with domain objects instead of external dependencies. The same class should be used to convert the `AddonNote` back to `Note` (so maybe we should keep track of the `note_id`
- [ ] Once we have that class, we should update the rest of the codebase accordingly (e.g., note formatter, note counter, etc.)
- [ ] ...